In [ ]:
import os
from os.path import join, isdir, isfile, abspath, dirname, splitext, basename, split

from IPython.display import Markdown, display
from dotenv import load_dotenv
import numpy as np
import pandas as pd

from misc_util.pretty_print import pretty_print as print
from derive_conceptualspace.pipeline import CustomContext, SnakeContext, load_envfiles
from derive_conceptualspace.settings import ENV_PREFIX, get_setting

from misc_util.logutils import setup_logging
from misc_util.pretty_print import display, pretty_print as print
from derive_conceptualspace.util.threedfigure import ThreeDFigure
from derive_conceptualspace.semantic_directions.cluster_names import get_name_dict
from derive_conceptualspace.pipeline import SnakeContext, load_envfiles, cluster_loader
from derive_conceptualspace.util.result_analysis_tools import getfiles_allconfigs
from derive_conceptualspace.util.desc_object import DescriptionList
from derive_conceptualspace.evaluate.shallow_trees import classify_shallowtree

pd.options.display.float_format = '{:.3f}'.format

## Analysis of different "measures of faithfulness of the separation"

In [ ]:
setup_logging()
load_envfiles("siddata")
configs, print_cnf = getfiles_allconfigs("featureaxes", verbose=True)

In [ ]:
sample_ctx = SnakeContext.loader_context(config=configs[0]) #dict(debug=False, min_words_per_desc=80, dcm_quant_measure="count")
descriptions, dcm, embedding, featureaxes = sample_ctx.load("pp_descriptions", "filtered_dcm", "embedding", "featureaxes")
dcm.show_info(descriptions=descriptions)

In [ ]:
%precision %.3f

def show_metrics(featureaxes, metric="kappa_rank2rank_onlypos_min", lamb1=0.5, lamb2=0.1):
    #I'm interested in:
    # *hat bin2bin/f_one ne hohe überschneidung mit anderen guten (kappa_rank2rank_onlypos_min, kappa_digitized_onlypos_2, kappa_count2rank_onlypos)
    # *was sind so die überschneidungen dieser guten 
    all_metrics = list(list(featureaxes["metrics"].values())[0].keys())
    print(all_metrics)
    one_metric = {k: v[metric] for k, v in featureaxes["metrics"].items()}
    one_metric = dict(sorted(one_metric.items(), reverse=True, key=lambda x : float('-inf') if np.isnan(x[1]) else x[1])) #https://stackoverflow.com/q/4240050/5122790
    t1 = {k: v for k, v in one_metric.items() if v >= lamb1}
    t2 = {k: v for k, v in one_metric.items() if v >= lamb2 and k not in t1.keys()}
    print(f"Lamb1: {lamb1} with {len(t1)} entries | Lamb2: {lamb2} with {len(t2)} entries | others: {len(one_metric)-len(t1)-len(t2)}")
    display(t1)
    print("="*50)
    display(t2)

def highlight_nonzero_max(data):
    #df.style.apply(highlight_nonzero_max, axis=0), https://stackoverflow.com/a/62639983/5122790
    #df.style.highlight_max(color='lightgreen', axis=0)
    return [f'font-weight: bold' if v == data.max() and v > 0 else '' for v in data]

def generate_comparertable(metrics, maxval=1.1, minval=0.5):
    all_metrics = list(list(metrics.values())[0].keys())
    t1_permetric = {}
    for metric in all_metrics:
        one_metric = {k: v[metric] for k, v in metrics.items()}
        one_metric = dict(sorted(one_metric.items(), reverse=True, key=lambda x : float('-inf') if np.isnan(x[1]) else x[1])) #https://stackoverflow.com/q/4240050/5122790
        t1 = {k: v for k, v in one_metric.items() if v >= minval and v < maxval}
        t1_permetric[metric] = set(t1.keys())
    t1_table = np.zeros([len(t1_permetric), len(t1_permetric)])
    for n1, k1 in enumerate(t1_permetric.values()):
        for n2, k2 in enumerate(t1_permetric.values()):
            t1_table[n1,n2] = len(k1&k2)/len(k1) if len(k1) > 0 else 0
    columns = [i.replace("kappa", "k").replace("dense", "d").replace("rank2rank", "r2r").replace("count2rank", "c2r").replace("bin2bin", "b2b").replace("f_one", "f1").replace("digitized", "dig").replace("_onlypos", "+") for i in t1_permetric.keys()]
    df = pd.DataFrame(t1_table, columns=columns, index=[f"{i} ({len(list(t1_permetric.values())[n])})" for n, i in enumerate(columns)])
    df.values[tuple([np.arange(df.shape[0])]*2)] = np.nan
    df = df.style.apply(highlight_nonzero_max, axis=1).format(precision=3)
    return df

In [ ]:
show_metrics(featureaxes, lamb1=0.3, lamb2=0.05)
#I'm interested in:
# *hat bin2bin/f_one ne hohe überschneidung mit anderen guten (kappa_rank2rank_onlypos_min, kappa_digitized_onlypos_2, kappa_count2rank_onlypos)
# *was sind so die überschneidungen dieser guten 

In [ ]:
generate_comparertable(sample_ctx.obj["featureaxes"]["metrics"], minval=0.5)
#look at combis of bin2bin, f_one, k_r2r+_min, k_dig+_2, k_c2r+

* bin2bin und f1 haben sehr hohe überschneidung (bin2bin ist aber strenger)
* k_r2r+_min and k_dig+_2 have the exact same results
* k_r2r+_min and k_c2r+ don't have too high of an overlap
* ALL of k_r2r+_min/k_dig+_2 (16), k_c2r+ (9) are in b2b
* ALL of the onlypos-statistics are completely in the respective kappa bin2bin

In [ ]:
generate_comparertable(sample_ctx.obj["featureaxes"]["metrics"], minval=0.1)

In [ ]:
generate_comparertable(sample_ctx.obj["featureaxes"]["metrics"], minval=0.1, maxval=0.5)